# Introduction to forward kinematics (i.e., direct geometry)

Do imports.

In [1]:
# Rigid body dynamics (pinocchio)
import pinocchio as pin

# Visualization (meshcat)
from pinocchio.visualize import MeshcatVisualizer
import meshcat_shapes

# Robot models (robot_descriptions)
from robot_descriptions.loaders.pinocchio import load_robot_description

# Math ("expm" is the matrix exponential function)
import numpy as np
from scipy.linalg import expm, logm

# Timing
import time

# Plots
import matplotlib.pyplot as plt

# Interaction
import ipywidgets as widgets
from IPython.display import display

# Suppress the display of very small numbers
np.set_printoptions(suppress=True)

Load a robot model and show how many joints there are.

In [2]:
# Load model
robot = load_robot_description(
    'iiwa14_description',           # name of robot model
    root_joint=None,                # fixed base
)

# Show number of joints
print(f'There are {robot.nq} joints.')

There are 7 joints.


Print a list of all frames.

In [13]:
print('FRAMES\n')
for frame in robot.model.frames:
    print(f' {frame.name}')

FRAMES

 universe
 base
 iiwa_base_joint
 iiwa_link_0
 iiwa_joint_1
 iiwa_link_1
 iiwa_joint_2
 iiwa_link_2
 iiwa_joint_3
 iiwa_link_3
 iiwa_joint_4
 iiwa_link_4
 iiwa_joint_5
 iiwa_link_5
 iiwa_joint_6
 iiwa_link_6
 iiwa_joint_7
 iiwa_link_7
 iiwa_joint_ee
 iiwa_link_ee_kuka
 tool0_joint
 iiwa_link_ee


Display robot in browser.

In [14]:
# Create a visualizer
vis = MeshcatVisualizer(robot.model, robot.collision_model, robot.visual_model)
robot.setVisualizer(vis, init=False)
vis.initViewer(open=True)
vis.loadViewerModel()

# Choose what to display
vis.displayFrames(False)
vis.displayVisuals(True)
vis.displayCollisions(False)

# Add our own frames to the visualizer because the default frames are hard to see
# - All frames
frames_to_show = [
    'iiwa_link_0',
    'iiwa_link_1',
    'iiwa_link_2',
    'iiwa_link_3',
    'iiwa_link_4',
    'iiwa_link_5',
    'iiwa_link_6',
    'iiwa_link_7',
    'iiwa_link_ee_kuka',
]
# # - Just the space frame and body frame
# frames_to_show = [
#     'iiwa_link_0',
#     'iiwa_link_ee_kuka',
# ]
for frame in frames_to_show:
    meshcat_shapes.frame(vis.viewer['frames/' + frame], opacity=1.0, axis_length=0.2)

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7007/static/


Put the robot at its "neutral" configuration.

In [15]:
# Get and show the neutral configuration (most likely all zeros)
q = pin.neutral(robot.model)
print(f'{q = }')

# Do forward kinematics
# - Compute the placement of all joint frames (modifies robot.data but not robot.model or q)
pin.forwardKinematics(robot.model, robot.data, q)
# - Compute the placement of all link frames (modifies robot.data but not robot.model)
pin.updateFramePlacements(robot.model, robot.data)

# Show the configuration in the visualizer
vis.display(q)
for frame in frames_to_show:
    frame_id = robot.model.getFrameId(frame)
    vis.viewer['frames/' + frame].set_transform(robot.data.oMf[frame_id].homogeneous)

q = array([0., 0., 0., 0., 0., 0., 0.])


Create sliders that allow us to change the configuration.

In [16]:
# Callback function that receives all 7 slider values
def on_slider_change(change):
    q = np.deg2rad(np.array([slider.value for slider in sliders]))
    pin.forwardKinematics(robot.model, robot.data, q)
    pin.updateFramePlacements(robot.model, robot.data)
    vis.display(q)
    for frame in frames_to_show:
        frame_id = robot.model.getFrameId(frame)
        vis.viewer['frames/' + frame].set_transform(robot.data.oMf[frame_id].homogeneous)

# Create 7 sliders
sliders = []
for i in range(7):
    slider = widgets.FloatSlider(
        value=0,
        min=-180,
        max=180,
        step=0.1,
        description=f'Joint {i+1}:',
        continuous_update=True,  # Only update on release
        orientation='horizontal',
        readout=True,
        readout_format='.1f',
        layout=widgets.Layout(width='600px')  # Set slider width
    )
    # Attach the same callback to each slider
    slider.observe(on_slider_change, names='value')
    sliders.append(slider)

# Display all sliders
display(widgets.VBox(sliders))

# Initialize robot
on_slider_change(None)

Define a function to do forward kinematics by product of exponentials.

In [ ]:
def to_skew(w):
    w_skew = np.zeros([3,3])
    w_skew[0,1] = -w[2]
    w_skew[1,0] = w[2]
    w_skew[0,2] = w[1]
    w_skew[2,0] = -w[1]
    w_skew[1,2] = -w[0]
    w_skew[2,1] = w[0]
    return w_skew

def hat(V):
    V_hat = np.zeros([4,4])
    V_hat[0:3, 0:3] = to_skew(V[0:3])
    V_hat[0:3,3] = V[3:6]
    return V_hat

def forward_kinematics_poe(theta):
    l = [0.1575, 0.2025, 0.2045, 0.2155, 0.1845, 0.2155, 0.0810, 0.0450]

    # FIXME
    a = [[0., 0., 1.],
         [0., 1., 0.],
         [0., 0., 1.],
         [0., -1., 0.],
         [0., 0., 1.],
         [0., 1., 0.],
         [0., 0., 1.]]
    p = [[0., 0., l[0]],
         [0., 0., l[0]+l[1]],
         [0., 0., l[0]+l[1]+l[2]],
         [0., 0., l[0]+l[1]+l[2]+l[3]],
         [0., 0., l[0]+l[1]+l[2]+l[3]+l[4]],
         [0., 0., l[0]+l[1]+l[2]+l[3]+l[4]+l[5]],
         [0., 0., l[0]+l[1]+l[2]+l[3]+l[4]+l[5]+l[6]]]
    M = [[1., 0., 0., 0.],
         [0., 1., 0., 0.],
         [0., 0., 1., l[0]+l[1]+l[2]+l[3]+l[4]+l[5]+l[6]+l[7]],
         [0., 0., 0., 1.]]

    T = M
    for a_i, p_i, theta_i in reversed(list(zip(a, p, theta))):
        S = np.concatenate((a_i, -to_skew(a_i) @ p_i))
        T = expm(hat(S) * theta_i) @ T

    return T

Define a function to print a homogeneous transformation matrix, rounding very small numbers to (positive) zero, so it is easy to compare two different matrices.

In [8]:
def show_homog(T, decimals=8, tol=1e-10):
    T = T.copy()
    T[np.abs(T) < tol] = 0.
    with np.printoptions(suppress=True):
        print(T)

Compare our forward kinematic solution to the one given by pinocchio.

In [9]:
# Sample a configuration at random
q = pin.randomConfiguration(robot.model)
 
# Do forward kinematics
# - Compute the placement of all joint frames (modifies robot.data but not robot.model or q)
pin.forwardKinematics(robot.model, robot.data, q)
# - Compute the placement of all link frames (modifies robot.data but not robot.model)
pin.updateFramePlacements(robot.model, robot.data)

# Show the configuration in the visualizer
vis.display(q)
for frame in frames_to_show:
    frame_id = robot.model.getFrameId(frame)
    vis.viewer['frames/' + frame].set_transform(robot.data.oMf[frame_id].homogeneous)

# Get pose of last link (pinocchio)
T_0_1_pin = robot.data.oMf[robot.model.getFrameId(frames_to_show[-1])].homogeneous

# Get pose of last link (by hand)
T_0_1_poe = forward_kinematics_poe(q)

# Print the two solutions
print('PINOCCHIO')
show_homog(T_0_1_pin)
print('')
print('OURS')
show_homog(T_0_1_poe)

# Check that the two solutions are the same
assert(np.allclose(T_0_1_pin, T_0_1_poe))

PINOCCHIO
[[-0.24394269 -0.96013815  0.13647964 -0.29307901]
 [ 0.84982194 -0.14383764  0.50706351 -0.69223068]
 [-0.46722011  0.23967783  0.85103461  0.52944611]
 [ 0.          0.          0.          1.        ]]

OURS
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]


AssertionError: 

Compare the computation time.

In [ ]:
# Arrays in which to store computation times
t_pin = []
t_poe = []

# Number of iterations
n = 10000

# Iterate
for i in range(n):
    # Sample a configuration
    q = pin.randomConfiguration(robot.model)

    # Measure the time to do forward kinematics with pinocchio (in nanoseconds)
    start_time = time.perf_counter_ns()
    pin.framesForwardKinematics(robot.model, robot.data, q)
    end_time = time.perf_counter_ns()
    t_pin.append(end_time - start_time)

    # Measure the time to do forward kinematics by hand (in nanoseconds)
    start_time = time.perf_counter_ns()
    T = forward_kinematics_poe(q)
    end_time = time.perf_counter_ns()
    t_poe.append(end_time - start_time)

# Show histograms
plt.hist(t_pin, bins=np.linspace(0, 500000, 101), label='pin')
plt.hist(t_poe, bins=np.linspace(0, 500000, 101), label='poe')
plt.legend()
plt.xlabel('nanoseconds')
plt.ylabel('count')
plt.show()

# Print mean values
print(f'Mean computation times')
print(f' pin: {np.mean(t_pin):8.0f} ns')
print(f' poe: {np.mean(t_poe):8.0f} ns')

## Velocity kinematics

Define a function to compute the adjoint matrix

$$\textrm{Ad}_T = \begin{bmatrix} R & 0 \\ [p]_\times R & R \end{bmatrix}.$$

In [ ]:
def adjoint(T):
    R = T[0:3, 0:3]
    p = T[0:3, 3]
    return np.block([[R, np.zeros((3, 3))],
                     [to_skew(p) @ R, R]])

Define other useful functions.

In [ ]:
def from_skew(w_skew):
    return np.array([w_skew[2, 1], w_skew[0, 2], w_skew[1, 0]])

def vee(V_hat):
    return np.concatenate([from_skew(V_hat[0:3, 0:3]), V_hat[0:3, 3]])

Define a function to compute the space Jacobian.

In [ ]:
def velocity_kinematics_poe(theta, space=True):
    if not space:
        raise Exception('This function currently only computes the space Jacobian,')
    
    # Define lengths
    l = [0.1575, 0.2025, 0.2045, 0.2155, 0.1845, 0.2155, 0.0810, 0.0450]

    # Define placement of each joint
    # FIXME
    a = [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]]
    p = [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]]
    
    # Initialize space Jacobian as all zeros
    J = np.zeros((6, len(theta)))

    # Compute the first column of space Jacobian, as well
    # as quantities we need to compute the next column
    s_i = np.concatenate((a[0], -to_skew(a[0]) @ p[0]))
    theta_i = theta[0]
    J[:, 0] = s_i
    Ad = adjoint(expm(hat(s_i) * theta_i))

    # Compute all the other columns of the space Jacobian
    for i in range(len(theta))[1:]:
        a_i = a[i]
        p_i = p[i]
        theta_i = theta[i]
        s_i = np.concatenate((a_i, -to_skew(a_i) @ p_i))
        J[:, i] = Ad @ s_i
        Ad = Ad @ adjoint(expm(hat(s_i) * theta_i))

    # Return the final result
    return J

Check that our space Jacobian matches what is computed by Pinocchio.

In [ ]:
# PIN
# - Do forward velocity kinematics
pin.computeJointJacobians(robot.model, robot.data)
# - Get the space jacobian <-- WARNING - Pinocchio assumes twists are (v, w) and not (w, v)
J_s_pin_vw = pin.getFrameJacobian(robot.model, robot.data, robot.model.getFrameId(frames_to_show[-1]), pin.WORLD)
# - Reorder the space jacobian to be consistent with an assumption that twists are (w, v)
J_s_pin = np.vstack((J_s_pin_vw[3:, :], J_s_pin_vw[:3, :]))

# POE
J_s_poe = velocity_kinematics_poe(q)

# Print shape of each Jacobian
print(f'shape of J_s_pin: {J_s_pin.shape}')
print(f'shape of J_s_poe: {J_s_poe.shape}')

# Check if the two results are the same
assert(np.allclose(J_s_pin, J_s_poe))

## Inverse kinematics (Jacobian transpose method)

Set up inverse kinematics problem.

In [ ]:
# Choose a desired pose
T_0_D = np.array([[1., 0., 0., 0.3],
                  [0., 1., 0., 0.4],
                  [0., 0., 1., 0.5],
                  [0., 0., 0., 1.]])

# Show desired pose in the browser window
meshcat_shapes.frame(vis.viewer['frames/' + 'desired_pose'], opacity=1.0, axis_length=0.2)
vis.viewer['frames/' + 'desired_pose'].set_transform(T_0_D)


Choose an initial configuration at random.

In [ ]:
q = pin.randomConfiguration(robot.model)

# Do forward kinematics
# - Compute the placement of all joint frames (modifies robot.data but not robot.model or q)
pin.forwardKinematics(robot.model, robot.data, q)
# - Compute the placement of all link frames (modifies robot.data but not robot.model)
pin.updateFramePlacements(robot.model, robot.data)

# Show the configuration in the visualizer
vis.display(q)
for frame in frames_to_show:
    frame_id = robot.model.getFrameId(frame)
    vis.viewer['frames/' + frame].set_transform(robot.data.oMf[frame_id].homogeneous)

Apply gradient descent (i.e., Jacobian transpose method).

In [ ]:
# Max number of iterations
max_iters = 1e5

# Step size
alpha = 0.1

# Tolerance
e_tol = 1e-4

# Array to store errors
e = []

# Current number of iterations
num_iters = 0

while True:
    # Do forward kinematics
    T_0_1_poe = forward_kinematics_poe(q)

    # Do velocity kinematics
    J_s_poe = velocity_kinematics_poe(q)

    # Get and store current error
    v_s_error = vee(logm(T_0_D @ np.linalg.inv(T_0_1_poe)))
    e_cur = np.linalg.norm(v_s_error)
    e.append(float(e_cur))

    # Check if stopping criterion is met
    if e_cur < e_tol:
        print(f'SUCCESS (in {num_iters} iterations)')
        break

    # Check if maximum number of iterations has been reached
    if num_iters > max_iters:
        print(f'FAILURE (exceeded {max_iters} iterations)')
        break
    
    # Do one step of gradient descent
    
    q += alpha * J_s_poe.T @ v_s_error

    # Increment number of iterations
    num_iters += 1

    # Do forward kinematics
    # - Compute the placement of all joint frames (modifies robot.data but not robot.model or q)
    pin.forwardKinematics(robot.model, robot.data, q)
    # - Compute the placement of all link frames (modifies robot.data but not robot.model)
    pin.updateFramePlacements(robot.model, robot.data)

    # Show the configuration in the visualizer
    vis.display(q)
    for frame in frames_to_show:
        frame_id = robot.model.getFrameId(frame)
        vis.viewer['frames/' + frame].set_transform(robot.data.oMf[frame_id].homogeneous)
    vis.viewer['frames/' + 'desired_pose'].set_transform(T_0_D)

Plot error as a function of iteration.

In [ ]:
plt.plot(e, linewidth=2)
plt.yscale('log')
plt.xlabel('number of iterations', fontsize=12)
plt.ylabel('error', fontsize=12)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.grid()
plt.show()